In [26]:
#Imports
import numpy as np
import sklearn
import websockets
import yfinance as yf

from sklearn.decomposition import NMF
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urljoin, urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#function signatures
from scipy.sparse import csr_matrix
from typing import Sequence, Tuple

import importlib, full_nmf_pipeline as pl
importlib.reload(pl)

<module 'full_nmf_pipeline' from 'C:\\Users\\wilda\\Desktop\\PIC 16B\\Homeworks\\full_nmf_pipeline.py'>

In [3]:
#start_date = '2020-01-01'
#end_date = '2025-11-20'

tech_tickers = ['NVDA', 'AAPL', 'MSFT', 'GOOG', 'GOOGL', 'AMZN', 'AVGO', 'META', 'TSM', 'TSLA', 'ORCL', '005930.KS', 'NFLX', 'ASML', 'PLTR', 'BABA', 'AMD', 'CSCO', 'IBM', 'SAP.DE', 'MU', 'SHOP', 'CRM']
finance_tickers = ['JPM','V', 'MA', 'BAC', 'MS', 'GS', 'AXP', 'WFC', 'BRK.B']
healthcare_tickers = ['LLY', 'JNJ', 'ABBV', 'UNH', 'MRK', 'ABT', 'TMO', 'NVS', 'PFE', 'MRNA']
realrestate_tickers = ['WELL', 'PLD', 'AMT', 'EQIX', 'SPG', 'DLR', 'O', 'PSA', 'CBRE', 'CCI']
utility_tickers = ['NEE', 'CEG', 'SO', 'DUK', 'AEP', 'SRE', 'VST', 'D', 'XEL', 'EXC']
industrial_tickers = ['GE', 'CAT', 'RTX', 'GEV', 'BA', 'UNP', 'ETN', 'HON', 'DE', 'PH']


In [4]:
df_yf = pl.scrape_articles_yf(url = pl.base_yf_url, n = 100)
df_yf

,Article Title,url
0,US stock futures mixed after Fed decision as O...,https://finance.yahoo.com/news/live/stock-mark...
1,"Fed cuts rates, signals it may be done for now",https://finance.yahoo.com/news/federal-reserve...
2,Oracle sinks as revenue falls short,https://finance.yahoo.com/news/oracle-stock-si...
3,Why tariff revenue just dipped for the first t...,https://finance.yahoo.com/news/tariff-revenue-...
4,Cisco shares finally top dot-com record after ...,https://finance.yahoo.com/news/cisco-shares-fi...
5,Trump: Fed should have 'at least doubled' cut,https://finance.yahoo.com/news/live/fed-meetin...
6,Apollo CEO Rowan: No need for another Fed cut,https://finance.yahoo.com/news/apollo-ceo-marc...
7,Trump: Warner Bros. deal should include sale o...,https://finance.yahoo.com/news/trump-says-warn...
8,"Carvana notches $33B, 12-day run ahead of S&P ...",https://finance.yahoo.com/news/carvana-track-r...
9,Coca-Cola names a company veteran as its new CEO,https://finance.yahoo.com/news/coca-cola-names...


In [ ]:
pl.configure_headless_browser()
yf_sel = pl.get_text_selenium(pl.base_yf_url)
yf_sel

In [4]:
pl.ready_for_nltk()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wilda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
prep_nmf_yf, corpus_yf, vectorizer_yf, index_map_yf = pl.full_pipeline(df_yf['url'], 1000, 5)
yf_topic_sequence = pl.apply_nmf_topic_models(prep_nmf_yf, 10, index_map_yf)

Topic 1: rate,cut,inflat,year,decis,offici,see,three,2026,next
Topic 2: bloomberg,trump,said,manag,rare,fund,fight,us,author,real
Topic 3: ai,stock,share,compani,analyst,earn,investor,billion,result,revenu
Topic 4: ai,data,year,intern,compani,infrastructur,call,last,custom,time
Topic 5: revenu,trump,billion,read,releas,presid,one,even,financ,month
Topic 6: report,trillion,2026,second,bank,june,look,think,start,bas
Topic 7: oper,said,market,includ,busi,compani,year,gener,challeng,sale
Topic 8: futur,rate,earn,report,ai,feder,wednesday,wall,street,us
Topic 9: yahoo,financ,cut,manag,rate,good,share,third,analyst,news
Topic 10: trump,deal,bloomberg,us,presid,think,includ,offic,compani,chief


In [6]:
#yfinance symbol for S&P 500
#ticker = '^GSPC'


price_yf = yf.download(tech_tickers[9], start='2025-07-01', end = '2025-12-02')

#prepare price_yf for merging 
price_yf_reset = price_yf.copy()
price_yf_reset = price_yf.reset_index()
price_yf_reset.columns = price_yf_reset.columns.get_level_values(0)
price_yf_reset = price_yf_reset.rename(columns={'Date': 'date'})
price_yf_reset

C:\Users\wilda\AppData\Local\Temp\ipykernel_1912\3825359289.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_yf = yf.download(tech_tickers[9], start='2025-07-01', end = '2025-12-02')
[*********************100%***********************]  1 of 1 completed


Price,date,Close,High,Low,Open,Volume
0,2025-07-01,300.709991,305.890015,293.209991,298.459991,145085700
1,2025-07-02,315.649994,316.829987,303.820007,312.630005,119483700
2,2025-07-03,315.350006,318.450012,312.760010,317.989990,58042300
3,2025-07-07,293.940002,296.149994,288.769989,291.369995,131177900
4,2025-07-08,297.809998,304.049988,294.350006,297.000000,103246700
...,...,...,...,...,...,...
102,2025-11-24,417.779999,421.720001,401.089996,402.170013,96806400
103,2025-11-25,419.399994,420.480011,405.950012,414.420013,71915600
104,2025-11-26,426.579987,426.940002,416.890015,423.950012,63463000
105,2025-11-28,430.170013,432.929993,426.200012,426.589996,36252900


In [7]:
df_api = pl.news_todate(2025, 'TSLA')
df_api

Fetching page 1 ...
Fetching page 2 ...
Fetching page 3 ...
Fetching page 4 ...
Fetching page 5 ...
Fetching page 6 ...
Fetching page 7 ...
Fetching page 8 ...
Fetching page 9 ...
Fetching page 10 ...


C:\Users\wilda\Desktop\PIC 16B\Homeworks\full_nmf_pipeline.py:282: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['date'] = pd.to_datetime(df['date'])


,title,news_url,date,sentiment,ticker
0,Elon Musk Sets Self-Driving Tesla Robotaxi Cou...,https://www.investors.com/news/tesla-stock-elo...,2025-12-10 08:33:05-05:00,Positive,TSLA
1,Tesla stock: why buying the hype could cost yo...,https://invezz.com/news/2025/12/10/tesla-stock...,2025-12-10 11:54:03-05:00,Negative,TSLA
2,Analyst Downgrade Puts Tesla Stock's Risk In F...,https://www.forbes.com/sites/greatspeculations...,2025-12-10 05:10:04-05:00,Negative,TSLA
3,Can Tesla Robotaxis Get Fully Driverless by th...,https://www.zacks.com/stock/news/2802206/can-t...,2025-12-10 10:31:12-05:00,Positive,TSLA
4,Morgan Stanley Downgrades Tesla: Should You Re...,https://www.zacks.com/stock/news/2802532/morga...,2025-12-10 14:40:53-05:00,Negative,TSLA
...,...,...,...,...,...
495,"Meet the ""Infinite Money Glitch"" That Could Se...",https://www.fool.com/investing/2025/10/26/meet...,2025-10-26 05:05:00-04:00,Neutral,TSLA
496,Here's Why I Think Cathie Wood Is Selling Tesl...,https://www.fool.com/investing/2025/10/26/here...,2025-10-26 09:49:00-04:00,Neutral,TSLA
497,Tesla Stock Is Up 219% Since 2020. Can Investo...,https://www.fool.com/investing/2025/10/26/tesl...,2025-10-26 11:05:00-04:00,Positive,TSLA
498,Nvidia Stock vs. Tesla Stock: Certain Wall Str...,https://www.fool.com/investing/2025/10/26/nvid...,2025-10-26 06:45:00-04:00,Positive,TSLA


In [8]:
df_api['news_url'] = df_api['news_url'].apply(pl.url_failed)
df_api

,title,news_url,date,sentiment,ticker
0,Elon Musk Sets Self-Driving Tesla Robotaxi Cou...,FAILED,2025-12-10 08:33:05-05:00,Positive,TSLA
1,Tesla stock: why buying the hype could cost yo...,FAILED,2025-12-10 11:54:03-05:00,Negative,TSLA
2,Analyst Downgrade Puts Tesla Stock's Risk In F...,FAILED,2025-12-10 05:10:04-05:00,Negative,TSLA
3,Can Tesla Robotaxis Get Fully Driverless by th...,https://www.zacks.com/stock/news/2802206/can-t...,2025-12-10 10:31:12-05:00,Positive,TSLA
4,Morgan Stanley Downgrades Tesla: Should You Re...,https://www.zacks.com/stock/news/2802532/morga...,2025-12-10 14:40:53-05:00,Negative,TSLA
...,...,...,...,...,...
495,"Meet the ""Infinite Money Glitch"" That Could Se...",https://www.fool.com/investing/2025/10/26/meet...,2025-10-26 05:05:00-04:00,Neutral,TSLA
496,Here's Why I Think Cathie Wood Is Selling Tesl...,https://www.fool.com/investing/2025/10/26/here...,2025-10-26 09:49:00-04:00,Neutral,TSLA
497,Tesla Stock Is Up 219% Since 2020. Can Investo...,https://www.fool.com/investing/2025/10/26/tesl...,2025-10-26 11:05:00-04:00,Positive,TSLA
498,Nvidia Stock vs. Tesla Stock: Certain Wall Str...,https://www.fool.com/investing/2025/10/26/nvid...,2025-10-26 06:45:00-04:00,Positive,TSLA


In [9]:
#Drop url's that did not render
mask = df_api['news_url'] == 'FAILED'
df_api = df_api.drop(df_api[mask].index)
df_api.reset_index(drop=True, inplace=True)
df_api

,title,news_url,date,sentiment,ticker
0,Can Tesla Robotaxis Get Fully Driverless by th...,https://www.zacks.com/stock/news/2802206/can-t...,2025-12-10 10:31:12-05:00,Positive,TSLA
1,Morgan Stanley Downgrades Tesla: Should You Re...,https://www.zacks.com/stock/news/2802532/morga...,2025-12-10 14:40:53-05:00,Negative,TSLA
2,"Tesla's New Bull Case: FSD, Brand Recovery, & ...",https://www.zacks.com/commentary/2802555/tesla...,2025-12-10 17:31:08-05:00,Positive,TSLA
3,"If You'd Invested $3,500 in Tesla 12 Years Ago...",https://www.fool.com/investing/2025/12/09/if-y...,2025-12-09 14:05:00-05:00,Positive,TSLA
4,Should You Buy Tesla While It's Below $500?,https://www.fool.com/investing/2025/12/09/shou...,2025-12-09 23:02:00-05:00,Negative,TSLA
...,...,...,...,...,...
243,TSLA vs. F: Which Stock Holds an Upper Hand Po...,https://www.zacks.com/stock/news/2777688/tsla-...,2025-10-27 10:31:37-04:00,Positive,TSLA
244,"Meet the ""Infinite Money Glitch"" That Could Se...",https://www.fool.com/investing/2025/10/26/meet...,2025-10-26 05:05:00-04:00,Neutral,TSLA
245,Here's Why I Think Cathie Wood Is Selling Tesl...,https://www.fool.com/investing/2025/10/26/here...,2025-10-26 09:49:00-04:00,Neutral,TSLA
246,Tesla Stock Is Up 219% Since 2020. Can Investo...,https://www.fool.com/investing/2025/10/26/tesl...,2025-10-26 11:05:00-04:00,Positive,TSLA


In [10]:
prep_NMF,corpus, vectorizer, index_map= pl.full_pipeline(df_api['news_url'], 2000, 6)

api_topic_sequence = pl.apply_nmf_topic_models(prep_NMF,30, index_map) #(shape num_docs x num_topics)

Topic 1: tesla,motley,fool,robotaxi,ev,ark,stock,growth,trillion,market
Topic 2: zack,rank,quick,report,stock,return,zackstrad,cooki,month,free
Topic 3: musk,sharehold,tesla,vote,packag,pay,plan,board,trillion,compens
Topic 4: proactiv,content,news,team,secur,advis,understand,site,publish,crypto
Topic 5: usd,look,fed,ema,level,bit,forecast,open,think,xrp
Topic 6: model,tesla,program,sale,departur,year,car,cybertruck,execut,vehicl
Topic 7: finbold,sign,log,crypto,pleas,submit,inform,agre,cryptocurr,password
Topic 8: techcrunch,startup,transport,strictlyvc,event,sean,meta,reportedli,video,media
Topic 9: chip,ai,design,musk,engin,tesla,said,ai5,samsung,post
Topic 10: burri,short,overvalu,palantir,dilut,ridicul,substack,in,michael,all
Topic 11: waymo,robotaxi,oper,ride,tesla,hail,permit,arizona,servic,uber
Topic 12: analyst,stock,street,wall,ai,rate,year,monday,share,stanley
Topic 13: revenu,margin,tesla,billion,year,quarter,growth,q3,earn,compani
Topic 14: versant,data,cnbc,media,inbox,ne

In [11]:
#Normalizing date columns
df_api['date']= pd.to_datetime(df_api['date'],utc=True).dt.tz_convert(None).dt.date #to convert to datetime64
price_yf_reset['date'] = pd.to_datetime(price_yf_reset['date']).dt.date


In [12]:
df_yf_api = pd.merge(price_yf_reset, df_api, on=['date'], how='inner')
df_yf_api

,date,Close,High,Low,Open,Volume,title,news_url,sentiment,ticker
0,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Tesla risks losing CEO Musk if $1 trillion pay...,https://www.cnbc.com/2025/10/27/tesla-musk-pay...,Neutral,TSLA
1,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Why Is Tesla Stock Soaring Today?,https://www.fool.com/investing/2025/10/27/why-...,Positive,TSLA
2,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,"With Near-Term Uncertainty Ahead, Should Inves...",https://www.fool.com/investing/2025/10/27/with...,Negative,TSLA
3,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,2 Charts That Show Why Tesla Is Still in Trouble,https://www.fool.com/investing/2025/10/27/2-ch...,Negative,TSLA
4,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Tesla's chair warns shareholders: Give Elon Mu...,https://www.businessinsider.com/tesla-chair-sh...,Negative,TSLA
...,...,...,...,...,...,...,...,...,...,...
179,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,This Artificial Intelligence Stock Could Be th...,https://www.fool.com/investing/2025/12/01/this...,Neutral,TSLA
180,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,Wall Street Continues to Underestimate the Gro...,https://www.investopedia.com/wall-street-conti...,Positive,TSLA
181,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,Michael Burry says Tesla is 'ridiculously over...,https://www.businessinsider.com/michael-burry-...,Negative,TSLA
182,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,BLBD or TSLA: Which Is the Better Value Stock ...,https://www.zacks.com/stock/news/2797900/blbd-...,Neutral,TSLA


In [17]:
#ensuring same shape
num_docs, num_topics = api_topic_sequence.shape
df_api= df_api.iloc[:num_docs].reset_index(drop=True)

#turning topics into dataframe
topic_cols = [f'topic_{i}' for i in range(total_topics)]
df_topics = pd.DataFrame(api_topic_sequence, columns = topic_cols)

df = pd.concat([df_api, df_topics], axis=1)
df

,title,news_url,date,sentiment,ticker,topic_0,topic_1,topic_2,topic_3,topic_4,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,Can Tesla Robotaxis Get Fully Driverless by th...,https://www.zacks.com/stock/news/2802206/can-t...,2025-12-10,Positive,TSLA,0.021405,3.439565e-01,2.364369e-06,2.210532e-05,3.052952e-03,...,1.525617e-03,6.800167e-07,2.421087e-08,1.734145e-08,5.925532e-08,4.227495e-09,2.017863e-24,3.243729e-03,3.278798e-05,5.158657e-05
1,Morgan Stanley Downgrades Tesla: Should You Re...,https://www.zacks.com/stock/news/2802532/morga...,2025-12-10,Negative,TSLA,0.018122,2.476982e-01,1.079733e-21,1.389391e-02,3.106381e-03,...,3.207566e-02,9.308146e-07,2.174844e-03,2.897855e-03,2.153116e-08,1.676013e-01,4.067833e-02,6.459968e-12,4.842209e-06,1.075335e-08
2,"Tesla's New Bull Case: FSD, Brand Recovery, & ...",https://www.zacks.com/commentary/2802555/tesla...,2025-12-10,Positive,TSLA,0.011344,3.085502e-01,8.078109e-11,2.557618e-05,2.678335e-21,...,6.014800e-05,2.058987e-20,1.873951e-17,2.394573e-04,1.170785e-16,1.351332e-03,1.557362e-19,2.973650e-06,4.546215e-05,1.737337e-07
3,"If You'd Invested $3,500 in Tesla 12 Years Ago...",https://www.fool.com/investing/2025/12/09/if-y...,2025-12-09,Positive,TSLA,0.072978,2.851463e-03,3.309297e-08,4.997757e-10,1.337124e-07,...,1.701078e-02,2.042784e-17,8.606406e-03,9.559548e-03,5.849916e-02,8.959325e-06,2.154884e-11,1.861004e-09,8.901668e-12,2.194573e-02
4,Should You Buy Tesla While It's Below $500?,https://www.fool.com/investing/2025/12/09/shou...,2025-12-10,Negative,TSLA,0.125204,1.124705e-03,1.973285e-02,4.506612e-09,4.184910e-02,...,1.121020e-02,1.102937e-04,1.729585e-10,2.076686e-02,5.972711e-02,6.429560e-11,6.862733e-10,2.246583e-06,4.115116e-03,2.584087e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,Investing in Tesla (TSLA)? Don't Miss Assessin...,https://www.zacks.com/stock/news/2777654/inves...,2025-10-27,Positive,TSLA,0.000818,2.606344e-01,1.858127e-03,1.931878e-09,4.356502e-12,...,5.418213e-03,2.545750e-10,4.895103e-12,2.942121e-04,2.392935e-14,2.718449e-01,2.394473e-02,4.749800e-12,2.573769e-04,1.533047e-06
241,S&P 500 Gains and Losses Today: Shares of Qual...,https://www.investopedia.com/s-and-p-500-gains...,2025-10-27,Positive,TSLA,0.189954,4.845324e-08,2.757251e-02,1.588242e-13,3.076681e-10,...,1.545731e-01,3.554905e-06,4.304461e-10,2.551750e-03,7.022995e-05,1.007925e-02,1.594693e-02,6.509541e-12,1.368915e-05,4.924185e-06
242,The Secret to Finding the Next Tesla Is Hiding...,https://www.fool.com/investing/2025/10/27/the-...,2025-10-27,Positive,TSLA,0.208510,7.481638e-05,1.770631e-32,1.744083e-17,1.248805e-04,...,3.014475e-53,1.392218e-12,2.264595e-02,2.607272e-02,1.080673e-02,3.038758e-26,2.088438e-11,5.698398e-24,2.429535e-02,1.048385e-29
243,TSLA vs. F: Which Stock Holds an Upper Hand Po...,https://www.zacks.com/stock/news/2777688/tsla-...,2025-10-27,Positive,TSLA,0.400184,2.962227e-08,9.098246e-13,8.592888e-06,6.095315e-09,...,1.099461e-16,9.395487e-13,3.482109e-03,3.136930e-13,2.228645e-14,1.274490e-02,8.406453e-03,1.146192e-07,4.896140e-07,7.753592e-05


In [18]:
price_yf_reset['date'] = pd.to_datetime(price_yf_reset['date']).dt.date
combined_df = pd.merge(price_yf_reset, df, on='date', how='inner')
combined_df

,date,Close,High,Low,Open,Volume,title,news_url,sentiment,ticker,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Tesla risks losing CEO Musk if $1 trillion pay...,https://www.cnbc.com/2025/10/27/tesla-musk-pay...,Neutral,TSLA,...,3.937361e-06,3.108219e-19,2.928373e-10,3.089256e-08,2.653325e-02,2.902562e-02,8.382976e-02,1.427701e-09,1.023042e-05,3.794798e-06
1,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Why Is Tesla Stock Soaring Today?,https://www.fool.com/investing/2025/10/27/why-...,Positive,TSLA,...,5.105459e-02,1.455375e-02,1.414963e-09,1.415817e-07,1.414021e-19,3.978036e-14,2.087340e-30,7.665038e-04,1.081992e-18,8.102121e-06
2,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,"With Near-Term Uncertainty Ahead, Should Inves...",https://www.fool.com/investing/2025/10/27/with...,Negative,TSLA,...,1.757033e-02,1.219210e-11,8.880104e-13,6.646418e-04,1.022454e-02,8.138719e-07,6.615553e-07,4.410201e-05,6.385344e-24,2.481033e-04
3,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,2 Charts That Show Why Tesla Is Still in Trouble,https://www.fool.com/investing/2025/10/27/2-ch...,Negative,TSLA,...,1.684047e-25,1.086446e-19,1.000069e-18,4.488617e-18,8.877253e-03,1.633759e-02,2.499091e-25,2.150853e-03,2.292759e-09,6.968664e-09
4,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Tesla's chair warns shareholders: Give Elon Mu...,https://www.businessinsider.com/tesla-chair-sh...,Negative,TSLA,...,5.700021e-11,4.751159e-12,1.143338e-08,5.170038e-11,2.561839e-02,5.403851e-20,1.709809e-02,5.424901e-17,1.118575e-10,2.997406e-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,This Artificial Intelligence Stock Could Be th...,https://www.fool.com/investing/2025/12/01/this...,Neutral,TSLA,...,2.260989e-03,1.522310e-24,2.051950e-10,9.897140e-41,9.327075e-40,4.715070e-26,3.134162e-11,4.575609e-18,7.750419e-28,3.302147e-23
180,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,Wall Street Continues to Underestimate the Gro...,https://www.investopedia.com/wall-street-conti...,Positive,TSLA,...,2.552290e-02,9.839300e-03,3.393182e-07,4.335672e-03,4.049545e-37,2.938839e-03,2.955578e-14,6.712003e-03,4.606586e-24,4.533104e-04
181,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,Michael Burry says Tesla is 'ridiculously over...,https://www.businessinsider.com/michael-burry-...,Negative,TSLA,...,1.646794e-20,1.606812e-21,5.993821e-05,2.240709e-09,3.970009e-02,1.594547e-16,1.046887e-16,1.014127e-14,5.563987e-08,9.145991e-04
182,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,BLBD or TSLA: Which Is the Better Value Stock ...,https://www.zacks.com/stock/news/2797900/blbd-...,Neutral,TSLA,...,6.650313e-03,1.331735e-03,1.824500e-02,1.135859e-08,3.146023e-02,2.276217e-03,5.894470e-03,1.208276e-02,4.655724e-02,5.056990e-03


In [20]:
sentiment = {'Negative' : -1, 'Neutral' : 0, 'Positive': 1}
combined_df['sentiment'] = combined_df['sentiment'].replace(sentiment)
combined_df

,date,Close,High,Low,Open,Volume,title,news_url,sentiment,ticker,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Tesla risks losing CEO Musk if $1 trillion pay...,https://www.cnbc.com/2025/10/27/tesla-musk-pay...,0,TSLA,...,3.937361e-06,3.108219e-19,2.928373e-10,3.089256e-08,2.653325e-02,2.902562e-02,8.382976e-02,1.427701e-09,1.023042e-05,3.794798e-06
1,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Why Is Tesla Stock Soaring Today?,https://www.fool.com/investing/2025/10/27/why-...,1,TSLA,...,5.105459e-02,1.455375e-02,1.414963e-09,1.415817e-07,1.414021e-19,3.978036e-14,2.087340e-30,7.665038e-04,1.081992e-18,8.102121e-06
2,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,"With Near-Term Uncertainty Ahead, Should Inves...",https://www.fool.com/investing/2025/10/27/with...,-1,TSLA,...,1.757033e-02,1.219210e-11,8.880104e-13,6.646418e-04,1.022454e-02,8.138719e-07,6.615553e-07,4.410201e-05,6.385344e-24,2.481033e-04
3,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,2 Charts That Show Why Tesla Is Still in Trouble,https://www.fool.com/investing/2025/10/27/2-ch...,-1,TSLA,...,1.684047e-25,1.086446e-19,1.000069e-18,4.488617e-18,8.877253e-03,1.633759e-02,2.499091e-25,2.150853e-03,2.292759e-09,6.968664e-09
4,2025-10-27,452.420013,460.160004,438.690002,439.980011,105867500,Tesla's chair warns shareholders: Give Elon Mu...,https://www.businessinsider.com/tesla-chair-sh...,-1,TSLA,...,5.700021e-11,4.751159e-12,1.143338e-08,5.170038e-11,2.561839e-02,5.403851e-20,1.709809e-02,5.424901e-17,1.118575e-10,2.997406e-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,This Artificial Intelligence Stock Could Be th...,https://www.fool.com/investing/2025/12/01/this...,0,TSLA,...,2.260989e-03,1.522310e-24,2.051950e-10,9.897140e-41,9.327075e-40,4.715070e-26,3.134162e-11,4.575609e-18,7.750419e-28,3.302147e-23
180,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,Wall Street Continues to Underestimate the Gro...,https://www.investopedia.com/wall-street-conti...,1,TSLA,...,2.552290e-02,9.839300e-03,3.393182e-07,4.335672e-03,4.049545e-37,2.938839e-03,2.955578e-14,6.712003e-03,4.606586e-24,4.533104e-04
181,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,Michael Burry says Tesla is 'ridiculously over...,https://www.businessinsider.com/michael-burry-...,-1,TSLA,...,1.646794e-20,1.606812e-21,5.993821e-05,2.240709e-09,3.970009e-02,1.594547e-16,1.046887e-16,1.014127e-14,5.563987e-08,9.145991e-04
182,2025-12-01,430.140015,433.660004,425.290009,425.320007,57463600,BLBD or TSLA: Which Is the Better Value Stock ...,https://www.zacks.com/stock/news/2797900/blbd-...,0,TSLA,...,6.650313e-03,1.331735e-03,1.824500e-02,1.135859e-08,3.146023e-02,2.276217e-03,5.894470e-03,1.208276e-02,4.655724e-02,5.056990e-03


In [33]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 42 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          80 non-null     datetime64[ns]
 1   Close         184 non-null    float64       
 2   High          184 non-null    float64       
 3   Low           184 non-null    float64       
 4   Open          184 non-null    float64       
 5   Volume        184 non-null    int64         
 6   title         184 non-null    object        
 7   news_url      184 non-null    object        
 8   sentiment     184 non-null    int64         
 9   ticker        184 non-null    object        
 10  topic_0       184 non-null    float64       
 11  topic_1       184 non-null    float64       
 12  topic_2       184 non-null    float64       
 13  topic_3       184 non-null    float64       
 14  topic_4       184 non-null    float64       
 15  topic_5       184 non-null    float64   

In [37]:
print(combined_df['date'].unique())

<DatetimeArray>
['2025-10-27 00:00:00',                 'NaT', '2025-10-28 00:00:00',
 '2025-10-29 00:00:00', '2025-10-30 00:00:00', '2025-10-31 00:00:00',
 '2025-11-03 00:00:00', '2025-11-02 00:00:00', '2025-11-04 00:00:00',
 '2025-11-05 00:00:00', '2025-11-06 00:00:00', '2025-11-07 00:00:00',
 '2025-11-10 00:00:00', '2025-11-11 00:00:00', '2025-11-13 00:00:00',
 '2025-11-14 00:00:00', '2025-11-18 00:00:00', '2025-11-19 00:00:00',
 '2025-11-20 00:00:00', '2025-11-21 00:00:00', '2025-11-24 00:00:00',
 '2025-11-25 00:00:00', '2025-11-28 00:00:00', '2025-12-01 00:00:00']
Length: 24, dtype: datetime64[ns]


# ML

In [23]:
feature_columns = [col for col in combine d_df.columns if col.startswith('topic_')] 
X = combined_df[feature_columns]
y = combined_df['sentiment']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize/train RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)  
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# See which topics matter most
importances = pd.DataFrame({'Feature': feature_columns, 'Importance': clf.feature_importances_})
print(importances.sort_values(by='Importance', ascending=False))

Accuracy: 0.5675675675675675
              precision    recall  f1-score   support

          -1       0.60      0.25      0.35        12
           0       0.50      0.29      0.36         7
           1       0.57      0.89      0.70        18

    accuracy                           0.57        37
   macro avg       0.56      0.47      0.47        37
weighted avg       0.57      0.57      0.52        37

     Feature  Importance
18  topic_18    0.076456
13  topic_13    0.060577
1    topic_1    0.046134
15  topic_15    0.043735
7    topic_7    0.042494
17  topic_17    0.035528
27  topic_27    0.034496
12  topic_12    0.033785
9    topic_9    0.033712
19  topic_19    0.033334
3    topic_3    0.032723
21  topic_21    0.032593
23  topic_23    0.032348
6    topic_6    0.031790
16  topic_16    0.030567
28  topic_28    0.030507
4    topic_4    0.030243
25  topic_25    0.029359
14  topic_14    0.028561
2    topic_2    0.028506
8    topic_8    0.028223
22  topic_22    0.027630
11  topic_11   

In [40]:
# Topic columns and features
topic_columns = [col for col in combined_df.columns if col.startswith('topic_')]
feature_columns = topic_columns + ['sentiment']

# Aggregate by date
daily_df = combined_df.groupby('date').agg({col: 'mean' for col in feature_columns} | {'movement': 'first'}).reset_index()

# Features and target
X = daily_df[feature_columns]
y = daily_df['movement']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
RFC = RandomForestClassifier(n_estimators=100, random_state=42)
RFC.fit(X_train, y_train)
y_pred = clf.predict(X_test)
    
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
    
importance = pd.DataFrame({'Feature': feature_columns,'Score': clf.feature_importances_}).sort_values(by='Score', ascending=False)
print(importance)

Accuracy: 0.8
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.50      0.67         2

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5

      Feature     Score
8     topic_8  0.114813
30  sentiment  0.089135
1     topic_1  0.080895
3     topic_3  0.068141
25   topic_25  0.046354
16   topic_16  0.045128
26   topic_26  0.044571
22   topic_22  0.044296
24   topic_24  0.038791
27   topic_27  0.037748
10   topic_10  0.037462
13   topic_13  0.032221
19   topic_19  0.032178
7     topic_7  0.031390
0     topic_0  0.027826
6     topic_6  0.025062
14   topic_14  0.021977
18   topic_18  0.021480
23   topic_23  0.020444
4     topic_4  0.020097
15   topic_15  0.019149
5     topic_5  0.015834
29   topic_29  0.015177
21   topic_21  0.013694
9     topic_9  0.012120
11   topic_11  0.010481
17   topic_